# N body problem

Some notes and tests for n-body problem solution

According to Newton's law a force of attraction between body $i$ and $j$ can be expressed as

$$
\mathbf{F}_{ij}
    = \frac{G m_i m_j}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\| ^ 2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

Resultant force acting on $i$ can be found when summing over all the interactions with $N-1$ bodies 

$$
\mathbf{F}_{i}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_i m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$

or in explicit differential form

$$
\frac{d^2\mathbf{r}_i}{dt^2}
    =  \sum_{j=1 \atop j \ne i}^N \frac{G m_j }{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|^2} \frac{\left(\mathbf{r}_j - \mathbf{r}_i\right)}{\left\| \mathbf{r}_j - \mathbf{r}_i\right\|}
$$




In [125]:
import numpy as np
from scipy.integrate import odeint
from typing import List, Dict, Tuple, Self
from itertools import combinations, chain
from matplotlib import pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation

G = 1

In [126]:
class Body:
    def __init__(
        self,
        m : float,
        r_0 : Tuple[float, float], 
        v_0 : Tuple[float, float],
        name: str = "body"
    ) -> None:
        self.m = m
        self.r_0 = r_0
        self.x_0 = r_0[0]
        self.y_0 = r_0[1]
        self.v_0 = v_0
        self.vx_0 = v_0[0]
        self.vy_0 = v_0[1]
        self.name = name

    def distance(self, body: Self) -> float:
        return np.sqrt((self.x_0 - body.x_0) ** 2 + (self.y_0 - body.y_0) ** 2)

earth = Body(
    m = 1, 
    r_0 = (1, 0), 
    v_0 = (0, np.sqrt(333000)),
    name = "earth"
)
sun = Body(
    m = 333000, 
    r_0 = (0, 0),
    v_0 = (0, 0),
    name = "sun"
)
pp = Body(
    m = 100000,
    r_0 = (1, 2),
    v_0 = (0, 0),
    name = "pp"
)

r_sun_earth = sun.distance(earth)
r_sun_earth

1.0

In [136]:
class System:
    def __init__(self, *bodies: Body) -> None:
        self.bodies = bodies
        self.bodies_dict = {body.name : body for body in bodies}
        self.interactions = self.calculate_interactions()

    def prepare_initial_data(self):
        positions = [[body.x_0, body.y_0] for body in self.bodies]
        velocities = [body.v_0 for body in self.bodies]
        accelerations = [[self.interactions[body]['a_x'], self.interactions[body]['a_y']] for body in self.bodies_dict]
        
        s_0 = list(chain(*positions+velocities))
        dsdt =  list(chain(*velocities + accelerations))
        return {"s_0" : s_0, "dsdt": dsdt}
   
    def get_all_unique_bodies_combinations(self):
        return list(combinations(self.bodies, 2))
    
    def calculate_interactions(self):
        epsilon_space = 1e-9
        accs = {body.name: {'a_x': 0, 'a_y': 0} for body in self.bodies}

        for body1, body2 in self.get_all_unique_bodies_combinations():
            r_x = body2.x_0 - body1.x_0
            r_y = body2.y_0 - body1.y_0
            distance = body1.distance(body2)
            if distance - epsilon_space < 0:
                continue

            accs[body1.name]['a_x'] += body2.m * r_x / distance ** 3
            accs[body1.name]['a_y'] += body2.m * r_y / distance ** 3
            accs[body2.name]['a_x'] -= body1.m * r_x / distance ** 3
            accs[body2.name]['a_y'] -= body1.m * r_y / distance ** 3

        return accs
    
    def _plot_system(self):
        for body_name, body in self.bodies_dict.items():
            plt.scatter(body.x_0, body.y_0, label = body_name)
        
        plt.xlabel("X")
        plt.ylabel("Y")
        plt.show()

    def derivatives(self, s, t):
        num_bodies = len(self.bodies)
        positions = np.reshape(s[:2 * num_bodies], (num_bodies, 2))
        velocities = np.reshape(s[2 * num_bodies:], (num_bodies, 2))
        accelerations = np.zeros_like(positions)

        for i, body1 in enumerate(self.bodies):
            for j, body2 in enumerate(self.bodies):
                if i != j:
                    r = positions[j] - positions[i]
                    distance = np.sqrt(np.dot(r, r))
                    if distance < 0.000001:
                        continue  # Avoid division by zero
                    acc = body2.m * r / distance**3
                    accelerations[i] += acc

        dsdt = np.concatenate((velocities, accelerations)).flatten()
        return dsdt

    def solve(self, t):
        initial_data = self.prepare_initial_data()
        s_0 = initial_data["s_0"]
        solution = odeint(self.derivatives, s_0, t)
        return solution
   
# test           
s = System(earth, sun, pp)
s.bodies

s.bodies_dict

{'earth': <__main__.Body at 0x7f0414667050>,
 'sun': <__main__.Body at 0x7f04146509d0>,
 'pp': <__main__.Body at 0x7f041607fad0>}